In [1]:
import os
DEVELOPMENT_MODE = False
# Detect if we're running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
except:
    IN_COLAB = False

# Install if in Colab
if IN_COLAB:
    %pip install transformer_lens
    %pip install circuitsvis
    # Install a faster Node version
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs  # noqa

# Hot reload in development mode & not running on the CD
if not IN_COLAB:
    from IPython import get_ipython
    ip = get_ipython()
    if not ip.extension_manager.loaded:
        ip.extension_manager.load('autoreload')
        %autoreload 2
        
IN_GITHUB = os.getenv("GITHUB_ACTIONS") == "true"

# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
# if IN_COLAB or not DEVELOPMENT_MODE:
#     pio.renderers.default = "colab"
# else:
#     pio.renderers.default = "notebook_connected"
pio.renderers.default = 'notebook_connected' #1回設定しておけばいい！
print(f"Using renderer: {pio.renderers.default}")

import circuitsvis as cv
# Testing that the library works
cv.examples.hello("keno")

# Import stuff
import torch
import torch.nn as nn
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import plotly.express as px

from jaxtyping import Float
from functools import partial

# import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, FactoredMatrix

torch.set_grad_enabled(False)

def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

Using renderer: notebook_connected


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tf_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
model = HookedTransformer.from_pretrained_no_processing(model_id, torch_dtype=torch.bfloat16, device_map="auto")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model meta-llama/Meta-Llama-3-8B-Instruct into HookedTransformer


In [3]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to('cuda')

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

tf_outputs = tf_model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,
    temperature=None,
    num_beams=1,
    top_p=None,
)
response = tf_outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,
    temperature=None,
    top_p=None,
)

response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Arrrr, me hearty! Me name be Captain Chat, the scurviest pirate chatbot to ever sail the Seven Seas! Me be here to swab the decks o' yer mind with me trusty responses, savvy? I be ready to hoist the Jolly Roger and set sail fer a swashbucklin' good time, matey! So, what be bringin' ye to these fair waters?


  0%|          | 0/256 [00:00<?, ?it/s]

Arrrr, me hearty! Me name be Captain Chat, the scurviest pirate chatbot to ever sail the Seven Seas! Me and me trusty parrot, Polly, be here to swab the decks and answer yer questions, matey!


In [4]:
messages = [
    {"role": "user", "content": "Hello, my name is Keno." * 3},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to('cuda')


llama3_logits, llama3_cache = model.run_with_cache(input_ids, remove_batch_dim=True)
llama3_str_tokens = [tokenizer.decode(token, skip_special_tokens=False) for token in input_ids[0]]

print("Layer 0 Head Attention Patterns:")
attention_pattern = llama3_cache["pattern", 0, "attn"]
cv.attention.attention_patterns(tokens=llama3_str_tokens, attention=attention_pattern)

Layer 0 Head Attention Patterns:


In [5]:
llama3_str_tokens = [tokenizer.decode(token, skip_special_tokens=False) for token in input_ids[0]]
llama3_str_tokens

['<|begin_of_text|>',
 '<|start_header_id|>',
 'user',
 '<|end_header_id|>',
 '\n\n',
 'Hello',
 ',',
 ' my',
 ' name',
 ' is',
 ' K',
 'eno',
 '.H',
 'ello',
 ',',
 ' my',
 ' name',
 ' is',
 ' K',
 'eno',
 '.H',
 'ello',
 ',',
 ' my',
 ' name',
 ' is',
 ' K',
 'eno',
 '.',
 '<|eot_id|>',
 '<|start_header_id|>',
 'assistant',
 '<|end_header_id|>',
 '\n\n']

In [6]:
few_shot_text = "Apple → Red\nLime → Green\nPlum →"
zero_shot_text = "Plum →"
input_ids = tokenizer(few_shot_text, return_tensors="pt").input_ids.to('cuda')

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

tf_outputs = tf_model.generate(
    input_ids,
    max_new_tokens=16,
    eos_token_id=terminators,
    do_sample=False,
    temperature=None,
    num_beams=1,
    top_p=None,
)
response = tf_outputs[0][input_ids.shape[-1]:]
print(few_shot_text)
print(tokenizer.decode(response, skip_special_tokens=True))
print("#################")

input_ids = tokenizer(zero_shot_text, return_tensors="pt").input_ids.to('cuda')
tf_outputs = tf_model.generate(
    input_ids,
    max_new_tokens=16,
    eos_token_id=terminators,
    do_sample=False,
    temperature=None,
    num_beams=1,
    top_p=None,
)
response = tf_outputs[0][input_ids.shape[-1]:]
print(zero_shot_text)
print(tokenizer.decode(response, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Apple → Red
Lime → Green
Plum →
 Purple
Orange → Orange
Grape → Purple
Cherry → Red

#################
Plum →

Plum is a type of stone fruit that is often used in baking and


In [7]:
tokenizer.decode(tf_outputs[0][input_ids.shape[-1]:][0])

'\n'

In [8]:
clean_prompt = "Apple → Red\nLime → Green\nPlum →"
corrupted_prompt = "Plum →"

clean_tokens = tokenizer(clean_prompt, return_tensors="pt").input_ids.to('cuda')
corrupted_tokens = tokenizer(corrupted_prompt, return_tensors="pt").input_ids.to('cuda')

def logits_to_logit_diff(logits, correct_answer=" Purple", incorrect_answer="\n"):
    # model.to_single_token maps a string value of a single token to the token index for that token
    # If the string is not a single token, it raises an error.
    correct_index = model.to_single_token(correct_answer)
    incorrect_index = model.to_single_token(incorrect_answer)
    return logits[0, -1, correct_index] - logits[0, -1, incorrect_index]

# We run on the clean prompt with the cache so we store activations to patch in later.
clean_logits, clean_cache = model.run_with_cache(clean_tokens)
clean_logit_diff = logits_to_logit_diff(clean_logits)
print(f"Clean logit difference: {clean_logit_diff.item():.3f}")

# We don't need to cache on the corrupted prompt.
corrupted_logits = model(corrupted_tokens)
corrupted_logit_diff = logits_to_logit_diff(corrupted_logits)
print(f"Corrupted logit difference: {corrupted_logit_diff.item():.3f}")

Clean logit difference: 10.250
Corrupted logit difference: -7.719


In [9]:
corrupted_tokens

tensor([[128000,   2169,    372,  11651]], device='cuda:0')

In [10]:
# We define a residual stream patching hook
# We choose to act on the residual stream at the start of the layer, so we call it resid_pre
# The type annotations are a guide to the reader and are not necessary
def residual_stream_patching_hook(
    resid_pre: Float[torch.Tensor, "batch pos d_model"],
    hook: HookPoint,
    position: int
) -> Float[torch.Tensor, "batch pos d_model"]:
    # Each HookPoint has a name attribute giving the name of the hook.
    clean_resid_pre = clean_cache[hook.name]
    resid_pre[:, position, :] = clean_resid_pre[:, -1, :]
    return resid_pre

test_prompt = "Corn →"
def logits_to_logit_diff(logits, correct_answer=" Yellow", incorrect_answer="\n"):
    # model.to_single_token maps a string value of a single token to the token index for that token
    # If the string is not a single token, it raises an error.
    correct_index = model.to_single_token(correct_answer)
    incorrect_index = model.to_single_token(incorrect_answer)
    return logits[0, -1, correct_index] - logits[0, -1, incorrect_index]
# We make a tensor to store the results for each patching run. We put it on the model's device to avoid needing to move things between the GPU and CPU, which can be slow.
test_tokens = tokenizer(test_prompt, return_tensors="pt").input_ids.to('cuda')
num_positions = len(test_tokens[0])
ioi_patching_result = torch.zeros((model.cfg.n_layers, num_positions), device=model.cfg.device)


for layer in tqdm.tqdm(range(model.cfg.n_layers)):
    position = test_tokens.shape[1] - 1
    # Use functools.partial to create a temporary hook function with the position fixed
    temp_hook_fn = partial(residual_stream_patching_hook, position=position)
    # Run the model with the patching hook
    patched_logits = model.run_with_hooks(test_tokens, fwd_hooks=[
        (utils.get_act_name("resid_pre", layer), temp_hook_fn)
    ])
    # Calculate the logit difference
    patched_logit_diff = logits_to_logit_diff(patched_logits).detach()
    print('layer ', layer, 'decoded', repr(tokenizer.decode(patched_logits[0, -1, :].argmax())), patched_logit_diff)
    # Store the result, normalizing by the clean and corrupted logit difference so it's between 0 and 1 (ish)
    ioi_patching_result[layer, -1] = (patched_logit_diff - corrupted_logit_diff)/(clean_logit_diff - corrupted_logit_diff)

  0%|          | 0/32 [00:00<?, ?it/s]

layer  0 decoded '\n' tensor(-8.1875, device='cuda:0', dtype=torch.bfloat16)
layer  1 decoded '\n' tensor(-5.5000, device='cuda:0', dtype=torch.bfloat16)
layer  2 decoded '\n' tensor(-5.6250, device='cuda:0', dtype=torch.bfloat16)
layer  3 decoded '\n' tensor(-4.8125, device='cuda:0', dtype=torch.bfloat16)
layer  4 decoded '\n' tensor(-4.5000, device='cuda:0', dtype=torch.bfloat16)
layer  5 decoded '\n' tensor(-3.9062, device='cuda:0', dtype=torch.bfloat16)
layer  6 decoded ' ' tensor(-2.2812, device='cuda:0', dtype=torch.bfloat16)
layer  7 decoded ' ' tensor(-0.7500, device='cuda:0', dtype=torch.bfloat16)
layer  8 decoded ' Wheat' tensor(-0.3125, device='cuda:0', dtype=torch.bfloat16)
layer  9 decoded ' Wheat' tensor(0.5000, device='cuda:0', dtype=torch.bfloat16)
layer  10 decoded ' Wheat' tensor(1.1875, device='cuda:0', dtype=torch.bfloat16)
layer  11 decoded ' Wheat' tensor(2.0625, device='cuda:0', dtype=torch.bfloat16)
layer  12 decoded ' Wheat' tensor(3., device='cuda:0', dtype=to

In [11]:
# Add the index to the end of the label, because plotly doesn't like duplicate labels
token_labels = [f"{token}_{index}" for index, token in enumerate(model.to_str_tokens(test_tokens))]
imshow(ioi_patching_result, x=token_labels, xaxis="Position", yaxis="Layer", title="Normalized Logit Difference After Patching Residual Stream on the IOI Task")

In [12]:
# max logit difference and layer index
max_logit_diff, max_layer = ioi_patching_result.max(dim=0)
max_logit_diff, max_layer

(tensor([0.0000, 0.0000, 0.8945], device='cuda:0'),
 tensor([ 0,  0, 16], device='cuda:0'))

In [13]:
max_layer_id = max_layer.max().item()

In [14]:
max_layer_id

16

In [15]:
temp_hook_fn = partial(residual_stream_patching_hook, position=position)
patched_logits = model.run_with_hooks(test_tokens, return_type="logits", fwd_hooks=[
        (utils.get_act_name("resid_pre", max_layer_id), temp_hook_fn)
    ])


In [16]:
tokenizer.decode(patched_logits[0, -1, :].argmax())

' Yellow'

In [38]:
def residual_stream_patching_hook(
    resid_pre: Float[torch.Tensor, "batch pos d_model"],
    hook: HookPoint,
    position: int
) -> Float[torch.Tensor, "batch pos d_model"]:
    # Each HookPoint has a name attribute giving the name of the hook.
    clean_resid_pre = clean_cache[hook.name]
    resid_pre[:, position, :] = clean_resid_pre[:, -1, :]
    return resid_pre

test_prompt = "Corn →"
test_prompt_tokens = tokenizer(test_prompt, return_tensors="pt").input_ids.to('cuda')
position = test_prompt_tokens.shape[1] - 1

# multi token prediction
max_new_tokens = 16
for _ in range(max_new_tokens):
    temp_hook_fn = partial(residual_stream_patching_hook, position=position)
    patched_logits = model.run_with_hooks(test_prompt_tokens, return_type="logits", fwd_hooks=[
            (utils.get_act_name("resid_pre", max_layer_id), temp_hook_fn)
        ])
    next_token = patched_logits[0, -1, :].argmax()
    print('decoded', repr(tokenizer.decode(next_token)), next_token)
    test_prompt_tokens = torch.cat([test_prompt_tokens, next_token.unsqueeze(0).unsqueeze(0)], dim=1)

tokenizer.decode(test_prompt_tokens[0])

decoded ' Yellow' tensor(26541, device='cuda:0')
decoded '\n' tensor(198, device='cuda:0')
decoded 'Yellow' tensor(48799, device='cuda:0')
decoded ' corn' tensor(14095, device='cuda:0')
decoded ' is' tensor(374, device='cuda:0')
decoded ' a' tensor(264, device='cuda:0')
decoded ' popular' tensor(5526, device='cuda:0')
decoded ' variety' tensor(8205, device='cuda:0')
decoded ' of' tensor(315, device='cuda:0')
decoded ' corn' tensor(14095, device='cuda:0')
decoded ' that' tensor(430, device='cuda:0')
decoded ' is' tensor(374, device='cuda:0')
decoded ' known' tensor(3967, device='cuda:0')
decoded ' for' tensor(369, device='cuda:0')
decoded ' its' tensor(1202, device='cuda:0')
decoded ' bright' tensor(10107, device='cuda:0')


'<|begin_of_text|>Corn → Yellow\nYellow corn is a popular variety of corn that is known for its bright'

In [37]:
from transformer_lens.past_key_value_caching import HookedTransformerKeyValueCache

def residual_stream_patching_hook(
    resid_pre: Float[torch.Tensor, "batch pos d_model"],
    hook: HookPoint,
    position: int
) -> Float[torch.Tensor, "batch pos d_model"]:
    # Each HookPoint has a name attribute giving the name of the hook.
    clean_resid_pre = clean_cache[hook.name]
    resid_pre[:, position, :] = clean_resid_pre[:, -1, :]
    return resid_pre

test_prompt = "Corn →"
test_prompt_tokens = tokenizer(test_prompt, return_tensors="pt").input_ids.to('cuda')
position = test_prompt_tokens.shape[1] - 1
past_kv_cache = HookedTransformerKeyValueCache.init_cache(
        model.cfg, model.cfg.device, test_prompt_tokens.shape[0]
    )
print(past_kv_cache.entries[0].past_keys.shape)
# multi token prediction
max_new_tokens = 16

for _ in range(max_new_tokens):
    temp_hook_fn = partial(residual_stream_patching_hook, position=position)
    patched_logits = model.run_with_hooks(test_prompt_tokens, return_type="logits", fwd_hooks=[
            (utils.get_act_name("resid_pre", max_layer_id), temp_hook_fn)
        ],
        past_kv_cache=past_kv_cache)
    print(past_kv_cache.entries[0].past_keys.shape)
    next_token = patched_logits[0, -1, :].argmax()
    print('decoded', repr(tokenizer.decode(next_token)), next_token)
    test_prompt_tokens = torch.cat([test_prompt_tokens, next_token.unsqueeze(0).unsqueeze(0)], dim=1)

tokenizer.decode(test_prompt_tokens[0])

torch.Size([1, 0, 8, 128])
torch.Size([1, 3, 8, 128])
decoded ' Yellow' tensor(26541, device='cuda:0')
torch.Size([1, 7, 8, 128])
decoded ' Corn' tensor(22036, device='cuda:0')
torch.Size([1, 12, 8, 128])
decoded '\n' tensor(198, device='cuda:0')
torch.Size([1, 18, 8, 128])
decoded 'Yellow' tensor(48799, device='cuda:0')
torch.Size([1, 25, 8, 128])
decoded ' Corn' tensor(22036, device='cuda:0')
torch.Size([1, 33, 8, 128])
decoded ' →' tensor(11651, device='cuda:0')
torch.Size([1, 42, 8, 128])
decoded ' Yellow' tensor(26541, device='cuda:0')
torch.Size([1, 52, 8, 128])
decoded ' Corn' tensor(22036, device='cuda:0')
torch.Size([1, 63, 8, 128])
decoded '\n' tensor(198, device='cuda:0')
torch.Size([1, 75, 8, 128])
decoded 'Yellow' tensor(48799, device='cuda:0')
torch.Size([1, 88, 8, 128])
decoded ' Corn' tensor(22036, device='cuda:0')
torch.Size([1, 102, 8, 128])
decoded ' →' tensor(11651, device='cuda:0')
torch.Size([1, 117, 8, 128])
decoded ' Yellow' tensor(26541, device='cuda:0')
torch.

'<|begin_of_text|>Corn → Yellow Corn\nYellow Corn → Yellow Corn\nYellow Corn → Yellow Corn\nYellow'

In [31]:
past_kv_cache.entries[0].past_keys.shape

torch.Size([1, 187, 8, 128])

In [33]:
model.to_tokens(test_prompt)

tensor([[128000,  91641,  11651]], device='cuda:0')

In [34]:
tokenizer(test_prompt, return_tensors="pt").input_ids.to('cuda')

tensor([[128000,  91641,  11651]], device='cuda:0')